# Tiền xử lý

In [1]:
import pandas as pd

# Đọc dữ liệu từ file csv
fall_dataset = pd.read_csv("data/data.csv")

# Xóa các hàng có giá trị bị thiếu hoặc trùng lặp
fall_dataset.drop_duplicates(inplace=True)
fall_dataset.dropna(inplace=True)

# Xóa các hàng có giá trị 0 quá 30%
fall_dataset = fall_dataset.loc[(fall_dataset == 0).mean(axis=1) <= 0.7]

In [2]:
fall_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,1161,1162,1163,1164,1165,1166,1167,1168,1169,label
0,0.479832,0.226308,-0.191834,0.522566,0.299079,0.029988,0.441129,0.297831,0.039410,0.520958,...,0.439730,0.641632,-0.022814,0.516509,0.778162,0.203044,0.433096,0.783251,0.180274,0
1,0.473642,0.218424,-0.244884,0.516687,0.285470,0.002815,0.432286,0.283625,0.011435,0.555427,...,0.437831,0.638412,-0.023069,0.496939,0.799569,0.106396,0.432793,0.775588,0.152766,0
2,0.492018,0.201748,-0.240479,0.535070,0.287116,0.011310,0.450817,0.278284,0.000268,0.553667,...,0.454501,0.649078,-0.035261,0.495688,0.793971,0.183968,0.462640,0.793851,0.168975,0
3,0.476379,0.211881,-0.186378,0.518921,0.283963,0.019227,0.437149,0.287399,0.036340,0.546644,...,0.430748,0.636678,-0.033758,0.498893,0.795711,0.078808,0.407627,0.780113,0.086856,0
4,0.483701,0.211692,-0.235794,0.526128,0.287072,0.043727,0.442634,0.289683,-0.014100,0.534582,...,0.454870,0.652144,-0.045462,0.502551,0.789684,0.178976,0.462765,0.791440,0.158383,0


In [3]:
fall_dataset["label"].value_counts()

label
0    120
1     34
Name: count, dtype: int64

In [4]:
# Tăng số lượng dữ liệu bằng cách nhân bản dữ liệu
for i in range(0, 5):
    fall_dataset = pd.concat([fall_dataset, fall_dataset])  # type: ignore

In [5]:
fall_dataset["label"].value_counts()

label
0    3840
1    1088
Name: count, dtype: int64

In [6]:
X = fall_dataset.drop(["label"], axis=1)
y = fall_dataset["label"]

In [7]:
from imblearn.over_sampling import SMOTE

# Áp dụng Smote để cần bằng dữ liệu bằng cách tạo thêm các mẫu ví dụ cho lớp thiểu số
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)  # type: ignore

In [8]:
y.value_counts()

label
0    3840
1    3840
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
# Đếm mỗi loại giá trị của mỗi lớp trong tập huấn luyện và tập kiểm tra
print("Số ví dụ trong tập train: %s" % y_train.count())
print("Số ví dụ trong tập test: %s" % y_test.count())

Số ví dụ trong tập train: 6144
Số ví dụ trong tập test: 1536


In [11]:
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa các đặc trưng (tùy chọn, nhưng có thể cải thiện hiệu suất cho một số mô hình)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [12]:
X_test

array([[-1.66655703, -1.30867195,  0.64523031, ...,  0.02441417,
         0.59329313,  0.80543481],
       [ 0.33226585, -0.29674055, -0.26971272, ...,  0.01505911,
         0.35597281,  0.64527707],
       [ 0.51994674, -0.13798492,  0.78166517, ...,  0.06483052,
         0.31275677, -1.74200346],
       ...,
       [ 0.33725953, -0.34419038, -0.38115963, ...,  0.02263999,
         0.3617603 ,  0.69741459],
       [ 1.81678656,  1.98157257,  0.54388775, ...,  1.41532431,
         0.44768647, -1.78859301],
       [ 1.45035689, -0.32545083,  1.48131412, ...,  2.23536164,
         0.64121996, -0.76514918]])

# Vẽ biểu đồ 3D

In [13]:
import plotly.graph_objects as go
import pandas as pd

data_sets = [(X_train, y_train, "Train"), (X_test, y_test, "Test")]
train_colors = ["blue", "red"]
test_colors = ["green", "orange"]
visible_dict = {"Normal": True, "Anomaly": True}
fig = go.Figure()

# Tạo biểu đồ 3D cho tập huấn luyện và tập kiểm tra
for (X, y, name_prefix), colors in zip(data_sets, [train_colors, test_colors]):
    balanced_data = pd.DataFrame(
        {"x": X[:, 0], "y": X[:, 1], "z": X[:, 2], "anomaly": y}
    ).sample(n=min(50000, len(X)), random_state=42)

    for anomaly_label, color in zip(range(2), colors):
        scatter = go.Scatter3d(
            x=balanced_data[balanced_data["anomaly"] == anomaly_label]["x"],
            y=balanced_data[balanced_data["anomaly"] == anomaly_label]["y"],
            z=balanced_data[balanced_data["anomaly"] == anomaly_label]["z"],
            mode="markers",
            marker=dict(size=4, color=color, opacity=0.1),
            name=f'{name_prefix} Data: {"Normal" if anomaly_label == 0 else "Anomaly"}',
            visible=visible_dict.get("Normal", True)
            if anomaly_label == 0
            else visible_dict.get("Anomaly", True),
        )
        fig.add_trace(scatter)

# Thêm nút để chuyển đổi giữa các lớp
buttons = [
    dict(
        label="Normal",
        method="update",
        args=[
            {
                "visible": [
                    True,
                    False,
                    True,
                    False,
                    visible_dict["Anomaly"],
                    visible_dict["Anomaly"],
                ]
            }
        ],
    ),
    dict(
        label="Anomaly",
        method="update",
        args=[
            {
                "visible": [
                    False,
                    True,
                    False,
                    True,
                    visible_dict["Normal"],
                    visible_dict["Normal"],
                ]
            }
        ],
    ),
]

# Cập nhật và hiển thị biểu đồ
fig.update_layout(updatemenus=[dict(type="buttons", showactive=False, buttons=buttons)])
fig.show()

# Thuật toán học có giám sát (Supervised Learning Algorithms)

In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

supervised_learning_models = {
    "KNN": KNeighborsClassifier(n_neighbors=10),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
}

In [15]:
from sklearn.model_selection import GridSearchCV

# Huấn luyện mô hình
train_models = {}
for name, classifier in supervised_learning_models.items():
    model = GridSearchCV(classifier, param_grid={})
    model.fit(X_train, y_train)
    train_models[name] = model

In [16]:
# Đánh giá mô hình
for name, model in train_models.items():
    # Tạo dự đoán trên tập kiểm tra X_test
    y_pred = model.predict(X_test)

    # So sánh dự đoán với giá trị từ tập kết quả y_test
    confusion_mat = pd.crosstab(
        y_test, y_pred, rownames=["Actual"], colnames=["Predicted"]
    )

    # Lấy các giá trị TP, TN, FP, FN từ confusion matrix
    TP, TN, FP, FN = (
        confusion_mat[1][1],
        confusion_mat[0][0],
        confusion_mat[0][1],
        confusion_mat[1][0],
    )

    # Tính toán các chỉ số đánh giá
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = (2 * precision * recall) / (precision + recall)

    # In kết quả
    print(f"{name}:")
    print(confusion_mat)
    print(
        f"Accuracy: {accuracy*100:.2f}%, Precision: {precision*100:.2f}%, Recall: {recall*100:.2f}%, F1: {f1*100:.2f}%"
    )
    print(f"Time: {model.cv_results_['mean_fit_time'][0]:.2f} (s)\n")

KNN:
Predicted    0    1
Actual             
0          782    0
1            0  754
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.12 (s)

Decision Tree:
Predicted    0    1
Actual             
0          782    0
1            0  754
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.55 (s)

Naive Bayes:
Predicted    0    1
Actual             
0          758   24
1            0  754
Accuracy: 98.44%, Precision: 100.00%, Recall: 96.92%, F1: 98.43%
Time: 0.22 (s)



In [17]:
# Lưu mô hình tốt nhất
import pickle

with open("../model/model.pkl", "wb") as file:
    pickle.dump(train_models["KNN"], file)